nodelete top 6 variables with >20 missing ratios. also delete MiscVal.

standardising variables

no modify skewness (by boxcox )

In [11]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import statsmodels.formula.api as smf
warnings.filterwarnings('ignore')
from scipy import stats
from scipy.stats import norm, skew
from sklearn.preprocessing import LabelEncoder
import statsmodels.api as sm
color = sns.color_palette() 
sns.set_style('darkgrid') 
pd.set_option('display.float_format', lambda x: '{:.3f}'.format(x)) #Limiting floats output to 3 decimal points


train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
test.drop("Id", axis = 1, inplace = True)

y, lmbda=stats.boxcox(train['SalePrice'])
train['BoxCoxPrice']=((train['SalePrice']**lmbda - 1) / lmbda)
train['LogPrice']=np.log(train['SalePrice'])
prices=['SalePrice','LogPrice','BoxCoxPrice']
print(lmbda)





ntrain = train.shape[0]
ntest = test.shape[0]
y_train = train.SalePrice.values
all_data = pd.concat((train, test)).reset_index(drop=True)
#all_data.drop(['SalePrice'], axis=1, inplace=True)
print("all_data size no delete bvariables is : {}".format(all_data.shape))





-0.0773275771249
all_data size no delete bvariables is : (2412, 81)


In [12]:
# PoolQC : data description says NA means "no pool"
all_data.loc[:, "PoolQC"] = all_data.loc[:, "PoolQC"].fillna("No")

# MiscFeature : data description says NA means "no misc feature"
all_data.loc[:, "MiscFeature"] = all_data.loc[:, "MiscFeature"].fillna("No")

# Alley : data description says NA means "no alley access"                        
all_data.loc[:, "Alley"] = all_data.loc[:, "Alley"].fillna("None") 

# Fence: data description says NA means "no fence"
all_data.loc[:, "Fence"] = all_data.loc[:, "Fence"].fillna("No")
# FireplaceQu : data description says NA means "no fireplace"
all_data.loc[:, "FireplaceQu"] = all_data.loc[:, "FireplaceQu"].fillna("No")

# LotFrontage : NA most likely means no lot frontage
all_data.loc[:, "LotFrontage"] = all_data.loc[:, "LotFrontage"].fillna(0)
# BsmtQual etc : data description says NA for basement features is "no basement"     
all_data.loc[:, "BsmtQual"] = all_data.loc[:, "BsmtQual"].fillna("No")        
all_data.loc[:, "BsmtCond"] = all_data.loc[:, "BsmtCond"].fillna("No")        
all_data.loc[:, "BsmtExposure"] = all_data.loc[:, "BsmtExposure"].fillna("No")
all_data.loc[:, "BsmtFinType1"] = all_data.loc[:, "BsmtFinType1"].fillna("No")
all_data.loc[:, "BsmtFinType2"] = all_data.loc[:, "BsmtFinType2"].fillna("No")
all_data.loc[:, "BsmtFullBath"] = all_data.loc[:, "BsmtFullBath"].fillna(0)
all_data.loc[:, "BsmtHalfBath"] = all_data.loc[:, "BsmtHalfBath"].fillna(0)


# GarageType etc : data description says NA for garage features is "no garage"
all_data.loc[:, "GarageType"] = all_data.loc[:, "GarageType"].fillna("No")
all_data.loc[:, "GarageFinish"] = all_data.loc[:, "GarageFinish"].fillna("No")
all_data.loc[:, "GarageQual"] = all_data.loc[:, "GarageQual"].fillna("No")
all_data.loc[:, "GarageCond"] = all_data.loc[:, "GarageCond"].fillna("No")
all_data.loc[:, "GarageYrBlt"] = all_data.loc[:, "GarageYrBlt"].fillna(0)  

# MasVnrType : NA most likely means no veneer
all_data.loc[:, "MasVnrType"] = all_data.loc[:, "MasVnrType"].fillna("None")
all_data.loc[:, "MasVnrArea"] = all_data.loc[:, "MasVnrArea"].fillna(0)

# Electrical: data description says NA means "no Electrical system"  
all_data.loc[:, "Electrical"] =all_data.loc[:, "Electrical"].fillna("No") 

In [13]:
all_data['MSSubClass'] = all_data['MSSubClass'].apply(str)

## Overall Cond (Ordinal): Rates the overall condition of the house

all_data['OverallCond'] = all_data['OverallCond'].astype(str)


all_data['YrSold'] = all_data['YrSold'].astype(str)
all_data['MoSold'] = all_data['MoSold'].astype(str)

all_data = all_data.replace({"MSSubClass" : {20 : "SC20", 30 : "SC30", 40 : "SC40", 45 : "SC45", 
                                       50 : "SC50", 60 : "SC60", 70 : "SC70", 75 : "SC75", 
                                       80 : "SC80", 85 : "SC85", 90 : "SC90", 120 : "SC120", 
                                       150 : "SC150", 160 : "SC160", 180 : "SC180", 190 : "SC190"},
                             
                             "OverallCond" : {1 :"Very Poor",2 :"Poor",3 :"Fair",4 :"Below Average",5 :"Average",
                                              6 :"Above Average",7 :"Good",8 :"Very Good",9 :"Excellent",
                                              10 :"Very Excellent"},
                             "YrSold": {2006 : "Year2006", 2007 : "Year2007", 2008 : "Year2008", 2009 : "Year2009",
                                         2010 :"Year2010"},
                             
                       "MoSold" : {1 : "Jan", 2 : "Feb", 3 : "Mar", 4 : "Apr", 5 : "May", 6 : "Jun",
                                   7 : "Jul", 8 : "Aug", 9 : "Sep", 10 : "Oct", 11 : "Nov", 12 : "Dec"}
                      })

In [14]:
cols = ('FireplaceQu', 'BsmtQual', 'BsmtCond', 'GarageQual', 'GarageCond', 
        'ExterQual', 'ExterCond','HeatingQC', 'PoolQC', 'KitchenQual', 'BsmtFinType1', 
        'BsmtFinType2', 'Functional', 'Fence', 'BsmtExposure', 'GarageFinish', 'LandSlope',
        'LotShape', 'PavedDrive', 'Street', 'Alley', 'CentralAir', 'MSSubClass', 'OverallCond', 
        'YrSold', 'MoSold')

for c in cols:
    lbl = LabelEncoder() 
    lbl.fit(list(all_data[c].values)) 
    all_data[c] = lbl.transform(list(all_data[c].values))

# shape        
print('Shape all_data: {}'.format(all_data.shape))

Shape all_data: (2412, 81)


In [15]:
all_data['TotalSF'] = all_data['TotalBsmtSF'] + all_data['1stFlrSF'] + all_data['2ndFlrSF']

In [16]:
numeric_feats = all_data.dtypes[all_data.dtypes != "object"].index


exclude = ['SalePrice','BoxCoxPrice', 'LogPrice']
predictors=[x for x in list(numeric_feats) if x not in exclude]

train = all_data[:ntrain]
test = all_data[ntrain:]

mu=train[predictors].mean()
sigma=train[predictors].std() 

train[predictors]=(train[predictors]-mu)/sigma
test[predictors]=(test[predictors]-mu)/sigma
all_data = pd.concat((train, test)).reset_index(drop=True) 



In [17]:
all_data = pd.get_dummies(all_data)
print(all_data.shape)

(2412, 224)


In [18]:
train_new4 = all_data[:ntrain]
test_new4 = all_data[ntrain:]
train_new4.head() 

,1stFlrSF,2ndFlrSF,3SsnPorch,Alley,BedroomAbvGr,BoxCoxPrice,BsmtCond,BsmtExposure,BsmtFinSF1,BsmtFinSF2,...,SaleType_CWD,SaleType_Con,SaleType_ConLD,SaleType_ConLI,SaleType_ConLw,SaleType_Oth,SaleType_VWD,SaleType_WD,Utilities_AllPub,Utilities_NoSewr
0,-0.739,-0.796,-0.109,0.043,0.216,7.749,0.308,0.610,0.507,-0.307,...,0,0,0,0,0,0,0,1,1,0
1,0.671,-0.796,-0.109,0.043,0.216,7.697,0.308,-0.308,1.546,-0.307,...,0,0,0,0,0,0,0,1,1,0
2,-0.042,1.264,-0.109,0.043,0.216,7.991,0.308,-0.308,-0.226,-0.307,...,0,0,0,0,0,0,0,1,1,0
3,-1.243,0.933,-0.109,3.867,0.216,7.836,0.308,0.610,0.397,-0.307,...,0,0,0,0,0,0,0,1,1,0
4,-0.876,1.305,-0.109,0.043,0.216,7.770,0.308,0.610,-1.046,-0.307,...,0,0,0,0,1,0,0,0,1,0


In [19]:
train_new4.to_csv("Train4.csv")
test_new4.to_csv("Test4.csv")